Below done so far:
- access Yummly API with "Search Recipes API Call"
- search for "chicken" recipes
- convert JSON into dicts and lists with .json() function
- extract data from converted JSON
    - "recipe" table 
    - "course" table
    - "ingredients" table
    - "flavors" table
- search Yummly metadata dictionary for valid search terms
- look at an individual recipe in detail with "Get Recipe API Call"


To do:
- identify if any info from the recipe_detail can be used
- come up with systematic way to pull recipes from API
    - one possibility:
- store data from API calls in SQL database / csvs

In [111]:
# imports

import requests
import json
import pandas as pd
import numpy as np

In [112]:
# ID and Key

app_id = 'e2b9bebc'
app_key = '4193215272970d956cfd5384a08580a9'

In [113]:
# URLs

base_url = 'http://api.yummly.com/v1/api/recipes?'
metadata_url = 'http://api.yummly.com/v1/api/metadata/'

In [114]:
# headers with yummly ID and Key

headers = {'X-Yummly-App-ID':'e2b9bebc', 'X-Yummly-App-Key':'4193215272970d956cfd5384a08580a9'}

In [115]:
# params

parameters = {'q':'veal', 'allowedCourse[]':'Main Dishes', 'maxResult': 100} 

# NOTE: maxResult can be 1,000, limiting to 100 for now

In [116]:
# Call API

response = requests.get(base_url, headers=headers, params=parameters)

In [117]:
# Check status code

response.status_code

200

In [118]:
# Convert JSON to python dictionaries and lists

veal = response.json()

In [119]:
# View type of object it is

type(veal)

dict

In [120]:
# View top level keys

response_keys = veal.keys()
response_keys

[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']

In [121]:
# The matches key has all the data in it - view second key-value pair in the dictionary

veal['matches'][1]

{u'attributes': {u'course': [u'Main Dishes'], u'cuisine': [u'Italian']},
 u'flavors': {u'bitter': 0.16666666666666666,
  u'meaty': 0.6666666666666666,
  u'piquant': 0.0,
  u'salty': 0.16666666666666666,
  u'sour': 0.3333333333333333,
  u'sweet': 0.16666666666666666},
 u'id': u'Veal-Piccata-440885',
 u'imageUrlsBySize': {u'90': u'http://lh5.ggpht.com/n07xBftSqjBDM-U6rK8Kk1UxdMLS-4bm63--evmpgBWrEiH3DS-ZSOhN-hHZ5Iqaj3LrqK-vF4tS_aQz6Qz2QA=s90-c'},
 u'ingredients': [u'veal cutlets',
  u'kosher salt',
  u'ground black pepper',
  u'flour',
  u'unsalted butter',
  u'olive oil',
  u'dry white wine',
  u'chicken stock',
  u'lemon',
  u'fresh lemon juice',
  u'capers',
  u'chopped parsley'],
 u'rating': 5,
 u'recipeName': u'Veal Piccata',
 u'smallImageUrls': [u'http://lh6.ggpht.com/a5jOB3gw37W2ujaXy-X6dkS90TOT496zmOWvf2eF0rBuRz3QIjkKjdGDbqZq20Gr8HIkbmLW8DIQuO0UYjHohQ=s90'],
 u'sourceDisplayName': u'Saveur',
 u'totalTimeInSeconds': 1800}

In [122]:
dinghy = {}
for item in veal['matches']:
    dinghy[item.get('id')] = item.get('ingredients')

In [123]:
dinghy

{u'Asian-Veal-Steaks-1409004': [u'olive oil',
  u'scallions',
  u'sugar',
  u'soy sauce',
  u'lime',
  u'veal cutlets',
  u'garlic cloves'],
 u'Bistro-Veal-Burgers-Allrecipes': [u'bread',
  u'milk',
  u'worcestershire sauce',
  u'eggs',
  u'basil leaves',
  u'fresh rosemary',
  u'pepper',
  u'ground veal',
  u'cheddar cheese',
  u'hamburger buns'],
 u'Breaded-Veal-Cutlet-754003': [u'veal cutlets',
  u'flour',
  u'grated parmesan cheese',
  u'eggs',
  u'parsley',
  u'salt',
  u'pepper',
  u'grated nutmeg',
  u'milk',
  u'butter',
  u'lemon'],
 u'Breaded-Veal-with-Spiced-Vegetables-1402443': [u'olive oil',
  u'sweet potatoes',
  u'zucchini',
  u'garlic',
  u'coriander seeds',
  u'ground cumin',
  u'butter',
  u'veal chops',
  u'baby spinach',
  u'lemon wedges'],
 u'Brown-Veal-Stock-440699': [u'veal bones',
  u'carrots',
  u'onions',
  u'celery',
  u'tomato paste',
  u'white wine',
  u'plum tomatoes',
  u'garlic',
  u'bay leaves',
  u'black peppercorns',
  u'parsley'],
 u'Caramelized-Veal

In [124]:
# Create lists to put data into

cuisine = []
rec_id = []
rating = []
image_size = []
rec_source = []
tot_time_sec = []

In [125]:
# flavors

# recipe_info_dict = {}
flavors_dict = {}
ingredients_dict = {}
courses_dict = {}

In [126]:
for item in veal['matches']:
    
    cuisine.append(item['attributes'].get('cuisine'))
    rec_id.append(item.get('id'))
    image_size.append(item.get('imageUrlsBySize').keys())
    rating.append(item.get('rating'))
    rec_source.append(item.get('sourceDisplayName'))
    tot_time_sec.append(item.get('totalTimeInSeconds'))
#     recipe_info_dict[item.get('id')] = 
    courses_dict[item.get('id')] = item['attributes'].get('course')
    flavors_dict[item.get('id')] = item.get('flavors')
    ingredients_dict[item.get('id')] = item.get('ingredients')

In [135]:
# create flavor_df

flavor_df = pd.DataFrame(flavors_dict).transpose()

In [136]:
# view flavor_df
flavor_df.head()

,bitter,meaty,piquant,salty,sour,sweet
Asian-Veal-Steaks-1409004,0.666667,0.666667,0,0.833333,0.5,0.166667
Bistro-Veal-Burgers-Allrecipes,0.666667,0.833333,0.833333,0.666667,0.166667,0.166667
Breaded-Veal-Cutlet-754003,0.666667,0.833333,0,0.833333,0.666667,0.166667
Breaded-Veal-with-Spiced-Vegetables-1402443,0.166667,0.333333,0,0.166667,0.833333,0.166667
Brown-Veal-Stock-440699,0.166667,0.166667,0.166667,0.166667,0.333333,0.166667


In [128]:
# Search for valid course terms

meta_search_course = requests.get('http://api.yummly.com/v1/api/metadata/course?', headers=headers)

In [129]:
# Search for valid holiday terms

meta_search_holiday = requests.get('http://api.yummly.com/v1/api/metadata/holiday?', headers=headers)

In [130]:
# Search for valid cuisine terms

meta_search_cuisine = requests.get('http://api.yummly.com/v1/api/metadata/cuisine?', headers=headers)

In [131]:
meta_search.status_code

200

In [132]:
# Can't do .json() for metadata response, instead do .text

meta_search_course.text

u'set_metadata(\'course\', [{"id":"course-Main Dishes","name":"Main Dishes","type":"course","description":"Main Dishes","searchValue":"course^course-Main Dishes","localesAvailableIn":["en-US"]},{"id":"course-Desserts","name":"Desserts","type":"course","description":"Desserts","searchValue":"course^course-Desserts","localesAvailableIn":["en-US"]},{"id":"course-Side Dishes","name":"Side Dishes","type":"course","description":"Side Dishes","searchValue":"course^course-Side Dishes","localesAvailableIn":["en-US"]},{"id":"course-Appetizers","name":"Appetizers","type":"course","description":"Appetizers","searchValue":"course^course-Appetizers","localesAvailableIn":["en-US"]},{"id":"course-Salads","name":"Salads","type":"course","description":"Salads","searchValue":"course^course-Salads","localesAvailableIn":["en-US"]},{"id":"course-Breakfast and Brunch","name":"Breakfast and Brunch","type":"course","description":"Breakfast and Brunch","searchValue":"course^course-Breakfast and Brunch","locales

In [133]:
# ind_recipe = requests.get('http://api.yummly.com/v1/api/recipe/French-Onion-Soup-The-Pioneer-Woman-Cooks-_-Ree-Drummond-41364?_app_id=e2b9bebc&_app_key=4193215272970d956cfd5384a08580a9')